Привет.

Возьмите датасет с отключением клиентов от телефонной компании.

Подготовьте данные для решения задачи.

Решите задачу с помощью нейронной сети, используя слои Dense как мы делали на уроке.

Ваша задача правильно написать “голову” сети и подобрать правильно loss функцию.

In [23]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, precision_score, \
              recall_score, f1_score

from keras.models import Sequential
from keras.layers import Dense

In [24]:
tf.random.set_global_generator(0) #не помогло...
tf.random.set_seed(0)             #тоже не помогло...
np.random.seed(0)

Прочитаем данные.

In [25]:
df_train = pd.read_csv("churn-bigml-80.csv")
df_test = pd.read_csv("churn-bigml-20.csv")

In [26]:
df_train.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


Используем знания из ДЗ_16 и дропнем ненужные колонки. Оставим колонку со штатом и попробуем с ней и без нее

In [27]:
for df in df_train, df_test:
    df.drop(['Number vmail messages', 
             'Total day minutes',
             'Total eve minutes',
             'Total night minutes',
             'Total intl minutes'], axis=1, inplace=True)

Cформируем перечни колонок в зависимости от типов.

In [28]:
numeric_features = df_train.select_dtypes(include=['int', 'float']).columns
categorical_features = df_train.select_dtypes(include=['object']).columns
target = df_train.columns[-1]

Обработаем колонку с таргетом.

In [29]:
for df in df_train, df_test:
    df["Churn"] = df["Churn"].map({True: 1, False: 0})

Соберем пайплайн для предобработки данных.

In [30]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(sparse=False))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

Сформируем входные и выходные данные для тренировки и теста.

In [31]:
x_train, y_train = df_train.iloc[:, :-1], df_train.iloc[:, -1:]
x_test, y_test = df_test.iloc[:, :-1], df_test.iloc[:, -1:]

Обучим пайплайн.

In [32]:
preprocessor.fit(x_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 Index(['Account length', 'Area code', 'Total day calls', 'Total day charge',
       'Total eve calls', 'Total eve charge', 'Total night calls',
       'Total night charge', 'Total intl calls', 'Total intl charge',
       'Customer service calls'],
      dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(sparse=False))]),
                                 Index(['State', 'International plan', 'Voice mail plan'], dtype='object'))])

Обработаем данные.

In [33]:
x_train = preprocessor.transform(x_train)
x_test = preprocessor.transform(x_test)

input_shape = x_train.shape[1]
output_shape = 2

Соберем модель

In [34]:
model = Sequential()

model.add(Dense(input_shape, activation="relu"))
model.add(Dense(input_shape//2, activation="relu"))
model.add(Dense(input_shape//4, activation="relu"))
model.add(Dense(output_shape, activation="sigmoid"))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [35]:
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data = (x_test, y_test))

Epoch 1/10
84/84 [==============================] - 2s 10ms/step - loss: 0.4058 - accuracy: 0.8545 - val_loss: 0.3567 - val_accuracy: 0.8576
Epoch 2/10
84/84 [==============================] - 0s 5ms/step - loss: 0.3270 - accuracy: 0.8567 - val_loss: 0.3232 - val_accuracy: 0.8591
Epoch 3/10
84/84 [==============================] - 0s 5ms/step - loss: 0.2841 - accuracy: 0.8710 - val_loss: 0.2935 - val_accuracy: 0.8666
Epoch 4/10
84/84 [==============================] - 0s 5ms/step - loss: 0.2532 - accuracy: 0.8965 - val_loss: 0.2802 - val_accuracy: 0.8801
Epoch 5/10
84/84 [==============================] - 0s 5ms/step - loss: 0.2241 - accuracy: 0.9149 - val_loss: 0.2673 - val_accuracy: 0.8951
Epoch 6/10
84/84 [==============================] - 0s 5ms/step - loss: 0.2000 - accuracy: 0.9272 - val_loss: 0.2663 - val_accuracy: 0.8891
Epoch 7/10
84/84 [==============================] - 0s 6ms/step - loss: 0.1797 - accuracy: 0.9407 - val_loss: 0.2651 - val_accuracy: 0.9025
Epoch 8/10
84/84 [=

In [36]:
predictions = model.predict(x_test)
pred = predictions.argmax(axis = 1)

21/21 [==============================] - 0s 2ms/step


In [37]:
precision_score(y_test, pred), recall_score(y_test, pred), f1_score(y_test, pred)

(0.631578947368421, 0.631578947368421, 0.631578947368421)

In [38]:
confusion_matrix(y_test, pred)

array([[537,  35],
       [ 35,  60]], dtype=int64)